In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Intensity_Annotation.xlsx", names=["Intensity", "Sentence"])

In [3]:
! pip3 install ekphrasis

  Using cached https://files.pythonhosted.org/packages/93/4b/52da6b1523d5139d04e02d9e26ceda6146b48f2a4e5d2abfdf1c7bac8c40/matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/4a/1c/6359be64e8301b84160f6f6f7936bbfaaa5e9a4eab6cbc681db07600b949/tqdm-4.45.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c9/dc/45cdef1b4d119eb96316b3117e6d5708a08029992b2fee2c143c7a0a5cc5/colorama-0.4.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/e4/a79c57e22d6d09bbeb5e8febb8cfa0fe10ede69eed9c3458d3ec99014e20/ujson-2.0.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/07/08/a549ba8b061005bb629b76adc000f3caaaf881028b963c2e18f811c6edc1/numpy-1.18.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https:

In [ ]:
intensity = df['Intensity']
sent = df['Sentence']

In [1]:
import pickle
normalized_sents = pickle.load(open("normalized_sents.pkl","rb"))
intensity = pickle.load(open("intensity.pkl","rb"))

In [2]:
print(len(normalized_sents))
print(len(intensity))

898
898


In [16]:
from nltk import word_tokenize
import string
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import random
import math

In [4]:
def remove_punctuations(tokens):
    table = str.maketrans('', '', string.punctuation) 
    ptokens = [w.translate(table) for w in tokens]     
    return ptokens

In [10]:
max1 = 0
for y in normalized_sents :
    words = word_tokenize(y)
    words = remove_punctuations(words)
    words = [s for s in words if s]
    if(len(words)>max1) :
        max1= len(words)

print(max1)
    

79


In [13]:
embeddings_dict = {}
with open("glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [41]:
values = []

for y in normalized_sents :
    words = word_tokenize(y)
    words = remove_punctuations(words)
    words = [s for s in words if s]
    embed = []
    for word in words :
        if(word in embeddings_dict) :
            embed.extend(list(embeddings_dict[word]))
        else :
            embed.extend(list(embeddings_dict['unk']))
    diff = 79-len(words)
    if(diff>0) :
        for i in range(diff) :
            embed.extend(list(np.zeros((100,))))
    values.append(embed)  

In [42]:
values = np.array(values)
intensity = np.array(intensity)

In [43]:
output = []
for x in intensity :
    if(x>=7) :
        output.append(1)
    else :
        output.append(0)

In [44]:
output = np.array(output)

In [45]:
print(values.shape)
print(output.shape)

(898, 7900)
(898,)


In [47]:
from sklearn.decomposition import PCA
reducer = PCA()
reducer.fit(values)
values = reducer.transform(values)

In [35]:
print(values.shape)

(898, 898)


In [36]:
from sklearn.model_selection import train_test_split

In [63]:
train_data, test_data, train_output, test_output = train_test_split(values, intensity, test_size=0.2, random_state=42)

In [64]:
params = {'C':[0.1, 0.5, 1, 2], 'fit_intercept':[True, False]}
modelin = LogisticRegression(solver='lbfgs', max_iter=1e4, n_jobs=5, random_state=1, warm_start=False, penalty='l2')
model = GridSearchCV(modelin, params, cv=2, n_jobs=5)
# model = LogisticRegression(solver='lbfgs',max_iter=1e4, n_jobs=5, random_state=1, warm_start=False, C=1, fit_intercept=True)
clf = model.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))
predictions = clf.predict(train_data)
print(classification_report(train_output, predictions))
print(model.best_params_)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00        14
           3       0.12      0.10      0.11        20
           4       0.19      0.21      0.20        24
           5       0.09      0.18      0.12        17
           6       0.12      0.06      0.09        31
           7       0.33      0.44      0.38        34
           8       0.21      0.19      0.20        31
           9       0.22      0.22      0.22         9

    accuracy                           0.19       180
   macro avg       0.14      0.16      0.15       180
weighted avg       0.18      0.19      0.18       180

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        32
           3       1.00      1.00      1.00        79
           4       1.00      1.00      1.00        81
           5       1.00 

/home/tanisha17116/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
params = {'C':[0.1, 0.5, 1, 2, 3]}
rbf_svc = svm.SVC(kernel='rbf', gamma='scale', max_iter=1e4, C=0.5 )
clf = rbf_svc.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00        20
           4       0.31      0.21      0.25        24
           5       0.18      0.41      0.25        17
           6       0.00      0.00      0.00        31
           7       0.21      0.09      0.12        34
           8       0.24      0.87      0.38        31
           9       0.00      0.00      0.00         9

    accuracy                           0.23       180
   macro avg       0.12      0.20      0.13       180
weighted avg       0.14      0.23      0.15       180



/home/tanisha17116/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
model = RandomForestClassifier(random_state=1, n_jobs=5, max_leaf_nodes=20)
clf = model.fit(train_data, train_output)
predictions = clf.predict(test_data)
print(classification_report(test_output, predictions))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00        20
           4       0.33      0.12      0.18        24
           5       0.17      0.18      0.17        17
           6       0.00      0.00      0.00        31
           7       0.14      0.29      0.19        34
           8       0.14      0.39      0.21        31
           9       0.00      0.00      0.00         9

    accuracy                           0.16       180
   macro avg       0.10      0.12      0.09       180
weighted avg       0.11      0.16      0.11       180



/home/tanisha17116/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
